# **SDXL - Base**

In [ ]:
pip install diffusers invisible_watermark transformers accelerate safetensors

In [ ]:
pip install xformers

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch


pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16, use_safetensors=True, variant="fp16").to("cuda")

# pipe.enable_xformers_memory_efficient_attention() # For torch < 2.0
# pipe.unet = torch.compile(pipe.unet, mode="max-autotune", fullgraph=True) # For torch > 2.0

# pipe.enable_model_cpu_offload()

In [ ]:
# Default - num_inference_steps=50, guidance_scale=5

prompt = '''capture a fine-detailed front profile of a 30 year old women walking at a busy street, 
    street side food stall, photo realistic, f/2.8'''

image = pipe(prompt=prompt, guidance_scale=15).images[0]
image

# **SDXL - Base + Refiner**

In [ ]:
from diffusers import DiffusionPipeline
import torch

base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16",
    use_safetensors=True).to("cuda")

refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
).to("cuda")

# base.enable_model_cpu_offload()
# refiner.enable_model_cpu_offload()

In [ ]:
prompt = '''capture a fine-detailed front profile of a 30 year old women walking at a busy street, 
    street side food stall, photo realistic, f/2.8'''

image = base(
    prompt=prompt,
    num_inference_steps=50,
    denoising_end=0.8,
    output_type="latent",
).images

image = refiner(
    prompt=prompt,
    num_inference_steps=50,
    denoising_start=0.8,
    image=image,
).images[0]
image

# **SDXL - lcm-LoRA-sdxl**

In [ ]:
pip install peft

In [ ]:
import torch
from diffusers import LCMScheduler, AutoPipelineForText2Image

model_id = "stabilityai/stable-diffusion-xl-base-1.0"
adapter_id = "latent-consistency/lcm-lora-sdxl"

pipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

pipe.load_lora_weights(adapter_id)
pipe.fuse_lora()


In [ ]:
# num_inference_steps=4, blurry, no creativity, requires facial prompts
# guidance_scale=0 or guidance_scale between 1.0 and 2.0

prompt = '''a 30 year old women sitting on a wooden and metal chair, fair skin, casual attire, 
    sunglasses, heels, white background, sideby tea table, photoshoot, photo realistic'''

image = pipe(prompt=prompt, num_inference_steps=4, guidance_scale=1).images[0]
image



# **SDXL lightning - UNet**

In [ ]:
# pip install huggingface_hub

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, EulerDiscreteScheduler
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

base = "stabilityai/stable-diffusion-xl-base-1.0"
repo = "ByteDance/SDXL-Lightning"
ckpt = "sdxl_lightning_8step_unet.safetensors" # Use the correct ckpt for your step setting!

# Load model.
unet = UNet2DConditionModel.from_config(base, subfolder="unet").to("cuda", torch.float16)
unet.load_state_dict(load_file(hf_hub_download(repo, ckpt), device="cuda"))
pipe = StableDiffusionXLPipeline.from_pretrained(base, unet=unet, torch_dtype=torch.float16, variant="fp16")
pipe.to("cuda")

pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")


In [ ]:
prompt = '''a 30 year old women sitting on a chair in sunlit park, fair skin, sunglasses,
    photoshoot, ultra-realistic photograph, fine detailed, extreme close up, 
    f/2.8 aperture, 35mm lens'''

image = pipe(prompt=prompt, num_inference_steps=8, guidance_scale=0.8).images[0]
image

# **SDXL lightning - LoRA**

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, EulerDiscreteScheduler
from huggingface_hub import hf_hub_download


base = "stabilityai/stable-diffusion-xl-base-1.0"
repo = "ByteDance/SDXL-Lightning"
ckpt = "sdxl_lightning_4step_lora.safetensors" # Use the correct ckpt for your step setting!

# Load model.
pipe = StableDiffusionXLPipeline.from_pretrained(base, torch_dtype=torch.float16, variant="fp16").to("cuda")
pipe.load_lora_weights(hf_hub_download(repo, ckpt))
pipe.fuse_lora()

pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")


In [ ]:
prompt = ''''a 30 year old women sitting on a wooden and metal chair,
    fair skin, casual attire, sunglasses, heels,
    white background, sideby wooden table, photoshoot, photo realistic'''

image = pipe(prompt=prompt, num_inference_steps=4, guidance_scale=0).images[0]

image

# **playground-v2.5-1024px-aesthetic**

In [ ]:
!pip install git+https://github.com/huggingface/diffusers
# diffusers >= 0.27.0 unofficial and currently in dev

In [ ]:
from diffusers import DiffusionPipeline, EDMDPMSolverMultistepScheduler
import torch

pipe = DiffusionPipeline.from_pretrained(
    "playgroundai/playground-v2.5-1024px-aesthetic",
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")

pipe.scheduler = EDMDPMSolverMultistepScheduler()


In [ ]:
prompt = '''A high resolution photorealistic fashion photograph of a beautiful model wearing
    a red backless gown standing under spotlights against a black background. 
    Shot from a low angle perspective with a shallow depth of field.'''

image = pipe(prompt=prompt, num_inference_steps=50, guidance_scale=3).images[0]
image

# **Stable Diffusion x4 Upscaler**

In [ ]:
from diffusers import StableDiffusionUpscalePipeline
import torch

upscaler = StableDiffusionUpscalePipeline.from_pretrained(
    "stabilityai/stable-diffusion-x4-upscaler", torch_dtype=torch.float16).to("cuda")


# from PIL import Image
# image = Image.open("/kaggle/working/women_earrings.png").convert('RGB')

prompt = '''a beautiful lady sitting on a wooden and metal chair, fair skin, casual attire, 
    sunglasses, heels, white background, sideby tea table, photoshoot, photo realistic, 4k, f/2.8'''

upscaled_image = upscaler(prompt=prompt, image=image).images[0]

print(upscaled_image.size)
upscaled_image

# **RealESRGAN Upscaler**

In [ ]:
# Upload the below git repository as dataset to the notebook input
# https://github.com/ai-forever/Real-ESRGAN

In [ ]:
import sys
sys.path.append("/kaggle/input/real-esrgan-1")

import time
import torch
from PIL import Image
import numpy as np
from RealESRGAN import RealESRGAN

# from PIL import Image
# image = Image.open("/kaggle/input/sample.png").convert('RGB')

start = time.time()

# Base x 2 = 1024 x 2 = 2048
scale = 2
weight_path = "weights/RealESRGAN_x2.pth"

# # Base x 4 = 1024 x 4 = 4096
# scale = 4
# weight_path = "weights/RealESRGAN_x4.pth"

# # Base x 8 = 1024 x 8 = 8192
# scale = 8
# weight_path = "weights/RealESRGAN_x8.pth"

model = RealESRGAN(torch.device('cuda'), scale=scale)
model.load_weights(weight_path, download=True)

upscaled_image = model.predict(image)

end = time.time()
print(upscaled_image.size, "time=%.2fs" % (end - start))
upscaled_image

In [ ]:
upscaled_image.save("sample_upscaled.png")